In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read csv file:
df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Repeat all the steps:
df.drop(['EIN', 'NAME'], axis = 1, inplace= True)
application_types_to_replace = ['T9', 'T13','T12','T2','T25','T14','T29','T15','T17']
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")
filter_out = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']
value_counts = df['CLASSIFICATION'].value_counts()
rest = [index for index in value_counts.index if index not in filter_out]
classifications_to_replace = rest
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
categorical_columns = df.select_dtypes('object').columns.to_list()
numerical_columns = df.select_dtypes(include=np.number).columns.to_list()

converted_df = pd.get_dummies(df[categorical_columns], dtype=int)
converted_df = converted_df.reset_index(drop=True)
numerical_df = df[numerical_columns].reset_index(drop=True)

combined_df = pd.concat([converted_df, numerical_df], axis=1)
combined_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,108590,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,1,5000,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,6692,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,142590,1


In [5]:
# split data set:
target = combined_df['IS_SUCCESSFUL']
features = combined_df.iloc[:, 0:-1]

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Model Optimisation Method 1: Adding more hidden layers

In [10]:
# let's try build 3 hidden layers this time. 

improved_model1 = tf.keras.models.Sequential([
    tf.keras.Input(shape=(43,), name='input_layer'),
    tf.keras.layers.Dense(units=33, activation='relu', name='first_hidden_layer'),
    tf.keras.layers.Dense(units=23, activation='relu', name ='second_hidden_layer'),
    tf.keras.layers.Dense(units=13, activation='relu', name ='third_hidden_layer'),
    tf.keras.layers.Dense(units=1, activation='sigmoid', name='output_layer')
])

# everything else is the same:
improved_model1.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.legacy.Adam(),
    metrics = ['accuracy']
)

improved_model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_hidden_layer (Dense)  (None, 33)                1452      
                                                                 
 second_hidden_layer (Dense  (None, 23)                782       
 )                                                               
                                                                 
 third_hidden_layer (Dense)  (None, 13)                312       
                                                                 
 output_layer (Dense)        (None, 1)                 14        
                                                                 
Total params: 2560 (10.00 KB)
Trainable params: 2560 (10.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
improved_model1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 805us/step - loss: 0.5766 - accuracy: 0.7136
Epoch 2/100
804/804 [==============================] - 1s 752us/step - loss: 0.5561 - accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 1s 751us/step - loss: 0.5532 - accuracy: 0.7290
Epoch 4/100
804/804 [==============================] - 1s 759us/step - loss: 0.5514 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 806us/step - loss: 0.5492 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 784us/step - loss: 0.5484 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 1s 773us/step - loss: 0.5474 - accuracy: 0.7333
Epoch 9/100
804/804 [==============================] - 1s 776us/step - loss: 0.5470 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 1s 770us/step - los

In [8]:
# Evaluate the model using the test data
model_loss, model_accuracy = improved_model1.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# not quite ok, it's actually worse than the model in the starter code.

268/268 - 0s - loss: 0.5608 - accuracy: 0.7296 - 249ms/epoch - 928us/step
Loss: 0.5608479380607605, Accuracy: 0.7295626997947693


# Model Optimization Method 2: Adjust the number of neurons in each layer and alter the aactivatinon function

In [9]:
# Now let's try adding more number of neurons per layer and adjust the activation function in each layer.

improved_model2 = tf.keras.models.Sequential([
    tf.keras.Input(shape=(43,), name='input_layer'),
    tf.keras.layers.Dense(units=86, activation='tanh', name='first_hidden_layer'),
    tf.keras.layers.Dense(units=50, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name ='second_hidden_layer'),
    tf.keras.layers.Dense(units=24, activation='relu', name ='third_hidden_layer'),
    tf.keras.layers.Dense(units=1, activation='sigmoid', name='output_layer')
])

improved_model2.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.legacy.Adam(),
    metrics = ['accuracy']
)

improved_model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_hidden_layer (Dense)  (None, 86)                3784      
                                                                 
 second_hidden_layer (Dense  (None, 50)                4350      
 )                                                               
                                                                 
 third_hidden_layer (Dense)  (None, 24)                1224      
                                                                 
 output_layer (Dense)        (None, 1)                 25        
                                                                 
Total params: 9383 (36.65 KB)
Trainable params: 9383 (36.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
improved_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 847us/step - loss: 0.5662 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 1s 867us/step - loss: 0.5536 - accuracy: 0.7270
Epoch 3/100
804/804 [==============================] - 1s 869us/step - loss: 0.5513 - accuracy: 0.7289
Epoch 4/100
804/804 [==============================] - 1s 841us/step - loss: 0.5492 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 1s 827us/step - loss: 0.5487 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 1s 881us/step - loss: 0.5474 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 828us/step - loss: 0.5468 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 1s 834us/step - loss: 0.5470 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 1s 838us/step - loss: 0.5461 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 1s 869us/step - l

In [12]:
model_loss, model_accuracy = improved_model2.evaluate(X_test_scaled, y_test)
print(f"Loss of model2: {model_loss}, Accuracy of model2: {model_accuracy}")

# still not good, also worse than the model in the Starter_code.

268/268 [==============================] - 0s 560us/step - loss: 0.5705 - accuracy: 0.7305
Loss of model2: 0.5704628825187683, Accuracy of model2: 0.7304956316947937


# Optimisation Method 3: Change the learning rate and apply more epochs (training longer)

In [13]:
# this time we try adjusting the optimizer's learning rate to 0.001 and make epochs = 200.

improved_model3 = tf.keras.models.Sequential([
    tf.keras.Input(shape=(43,), name='input_layer'),
    tf.keras.layers.Dense(units=86, activation='tanh', name='first_hidden_layer'),
    tf.keras.layers.Dense(units=50, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name ='second_hidden_layer'),
    tf.keras.layers.Dense(units=24, activation='relu', name ='third_hidden_layer'),
    tf.keras.layers.Dense(units=1, activation='sigmoid', name='output_layer')
])

improved_model3.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

improved_model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_hidden_layer (Dense)  (None, 86)                3784      
                                                                 
 second_hidden_layer (Dense  (None, 50)                4350      
 )                                                               
                                                                 
 third_hidden_layer (Dense)  (None, 24)                1224      
                                                                 
 output_layer (Dense)        (None, 1)                 25        
                                                                 
Total params: 9383 (36.65 KB)
Trainable params: 9383 (36.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
improved_model3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 858us/step - loss: 0.5657 - accuracy: 0.7224
Epoch 2/200
804/804 [==============================] - 1s 868us/step - loss: 0.5549 - accuracy: 0.7289
Epoch 3/200
804/804 [==============================] - 1s 847us/step - loss: 0.5515 - accuracy: 0.7296
Epoch 4/200
804/804 [==============================] - 1s 889us/step - loss: 0.5505 - accuracy: 0.7311
Epoch 5/200
804/804 [==============================] - 1s 833us/step - loss: 0.5493 - accuracy: 0.7318
Epoch 6/200
804/804 [==============================] - 1s 938us/step - loss: 0.5480 - accuracy: 0.7324
Epoch 7/200
804/804 [==============================] - 1s 845us/step - loss: 0.5473 - accuracy: 0.7329
Epoch 8/200
804/804 [==============================] - 1s 878us/step - loss: 0.5468 - accuracy: 0.7319
Epoch 9/200
804/804 [==============================] - 1s 832us/step - loss: 0.5465 - accuracy: 0.7338
Epoch 10/200
804/804 [==============================] - 1s 828us/step - l

In [15]:
model_loss, model_accuracy = improved_model3.evaluate(X_test_scaled, y_test)
print(f"Loss of model3: {model_loss}, Accuracy of model3: {model_accuracy}")
# no improvement compared to the last one.

268/268 [==============================] - 0s 575us/step - loss: 0.6061 - accuracy: 0.7291
Loss of model3: 0.6060855388641357, Accuracy of model3: 0.7290962338447571


In [16]:
improved_model3.save("AlphabetSoupCharity_Optimisation.h5")

/opt/anaconda3/envs/dev2/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
